# Preparations at home

## Installation of the package

 1. (Only needed on Windows) Install the Windows Subsystem for Linux following [these instructions](https://learn.microsoft.com/en-us/windows/wsl/setup/environment). Then execute the following steps (including for starting the notebook) within a wsl environent. You can start one by typing `wsl` in a Powershell.
 2. To install a micromamba instance (if no mamba/conda is available) with all the dependencies in a dedicated environment, there is a bash script available. Take a look at the script at the script at the location that is downloaded *via* `wget` and confirm it is safe. Then download and execute the installation script by executing:

    ```bash    
        wget https://github.com/Niolon/XHARPy/raw/refs/heads/main/install.sh 
        bash install.sh
        
    ```
    Do **not** execute via `sudo`, doing that potentially makes your conda environments unusable.

## Starting the notebook 
You can start the notebook by opening a command line / shell in the folder where you have executed the command above (on windows use the WSL shell). Then type

```bash
conda activate xharpy
jupyter lab
```
After a short while a new website should open up. If if does not, open the link in the console output. Within the Jupyter UI, navigate to `Examples/Workshop_Erice` and open the `Workshop.ipynb` You should see an identical copy to these lecture notes as an interactive Jupyter notebook.

## Test the installation
Within the interactive version, you can execute the following cells by clicking on the cell and pressing `Shift + Enter`. If all of the tests show a pass: Congratulations you are ready for the workshop and have a running version of XHARPy on your computer.




In [ ]:
from workshop_tests import test_xharpy_installation
test_xharpy_installation();

# Introduction: What does XHARPy do?

 - Hirshfeld Atom Refinement (with partitioned valence densities)
 - atomic form factor calculation
 - Doing these things with periodic densities evaluated on rectangular grids with separately evaluated frozen core densities

# Prerequisites

## The two minute introduction to python programming

As you might imagine this is not a python course, just a short glossary of the syntax that will be used.

Values from the right are assigned to a variable on the left with the `=` operator

In [ ]:
a = 1

To save text we need to start and end with the same type of quotation mark

In [ ]:
a_string = "String content"

A dictionary is started and ended with curly braces and pairs of key: values separated by a colon. They are used for settings in this tutorial if you know how to write JSON you can basically transfer what you know.

In [ ]:
my_dict = {"setting_name": 1, "another_setting_name": "another value"}

we access / assign / modify a value in a dict using square brackets

In [ ]:
b = my_dict["setting_name"]
my_dict["a_third_setting"] = True

A function has a name and round brackets, in which you input arguments by position and/or by name. It can produce one or more *return values* that can be assigned to variables

In [ ]:
print("Output")  # no return value

In [ ]:
result = dict(name="Alice", age=30, city="New York")
print(result)  # we can use variables as arguments

We get more functionality / functions by importing them from installed packages

In [ ]:
import shutil
import numpy as np
from pathlib import Path

# Using XHARPy
First we will import the functions needed to road the data, execute the refinement and write the results back to disk. At the first time you import something you should get a warning about JAX using the CPU. This is expected (otherwise you need a 64 bit capable GPU, and refinement on the GPU is untested).

In [ ]:
# import functions for reading the intensities
from xharpy import shelxl_hkl2pd  # For a SHELX HKL 4 (that needs to be merged!)
from xharpy import xd_hkl2pd  # For a XD hkl containing intensities
from xharpy import fcf2hkl_pd # For reading the observed intensities from an fcf4 (do not correct the extinction against the IAM).

# import function to read other data
from xharpy import cif2data, lst2constraint_dict

# import functions needed for refinement
from xharpy import create_construction_instructions, refine

# import function to write out the results
from xharpy import write_cif, write_res, write_fcf, add_density_entries_from_fcf

# import functions to create a tsc file
from xharpy import cif2tsc

# Path is a really convenient way to work with pathes in python. (should already be present from example above)
from pathlib import Path

All functions within XHARPy should have a working in-function documentation (so a docstring), which you can access via python's help function.

In [ ]:
help(refine)

## Refining with Quantum Espresso: CaF<sub>2</sub>
Note that there is no logic between the assignment of calculation program for the density used in this tutorial: The aim is two show the two backend-engines and two different structures.

Let us start to work on the first dataset by creating references to the folders needed

In [ ]:
folder_caf2 = Path("CaF2")
output_folder_caf2 = folder_caf2 / "xharpy_output"
output_folder_caf2.mkdir(exist_ok=True)

### Loading the data and merging the hkl
There is a lot of different objects that will be created from the cif file using the `cif2data` function.

In [ ]:
atom_table_caf2, cell_caf2, cell_esd_caf2, symm_mats_vecs_caf2, symm_strings_caf2, wavelength_caf2 = cif2data(folder_caf2 / "CaF2.cif", 0)

First it is instructive to take a quick look at the imported `atom_table_caf2`. This an object which has a nice representation in the Jupyter notebook (for reference: it is a pandas DataFrame).

In [ ]:
atom_table_caf2

We can load and merge the hkl, check that the space group is correct. XHARPy will execute with unmerged reflections. However the refinement will get slow and the result will not be correct.

In [ ]:
hkl_caf2 = fcf2hkl_pd(folder_caf2/  'CaF2.fcf')

The final file, we need is an SHELXL lst file for the special position constraints. These can also be generated manually (as described [here](https://xharpy.readthedocs.io/en/latest/library/library_symm_con.html)). However creating them from an lst is more convenient. For reference, this is what this function is looking for. It might be possible to recreate this without ShelXl (make sure to create the spaces around the `*` signs): 
```
 Special position constraints for Ca01
 x =  0.5000   y =  0.5000   z =  0.5000   U22 = 1.0 * U11   U33 = 1.0 * U11   
 U23 = 0   U13 = 0   U12 = 0   sof = 0.02083   
 Input constraints retained (at least in part) for  sof

 Special position constraints for F002
 x =  0.2500   y =  0.7500   z =  0.7500   U22 = 1.0 * U11   U33 = 1.0 * U11   
 U23 = 0   U13 = 0   U12 = 0   sof = 0.04167   
 Input constraints retained (at least in part) for  sof
```

In [ ]:
constraints_caf2 = lst2constraint_dict(folder_caf2 / 'CaF2.lst')

### Setting up the refinement
The `refinement_dict` is used to control the refinement on a macroscopic scale. Available options can be found in the docstring of `refine` or in the [online documentation](https://xharpy.readthedocs.io/en/latest/library/library_refinement_dict.html). 

Settings that can be changed:
  - `f0j_source`:
    
    Source of the atomic form factors. The computation_dict 
    will be passed on to this method. See the individual files in
    f0j_sources for more information, by default `'gpaw'`
    Tested options: `'gpaw'`, `'iam'`, `'gpaw_mpi'`, `'qe'`, `'tsc_file'`
    Experimental options: `'nosphera2_orca'`, `'custom_function'`
    
  - `reload_step`:
    
    Starting with this step the computation will try to reuse the 
    density, if this is implemented in the source, by default 1
    
  - `core`:
    
    If this is implemented in a f0j_source, it will integrate the 
    frozen core density on a spherical grid and only use the valence
    density for the updated atomic form factos options are 
    'combine', which will not treat the core density separately,
    'constant' which will integrate and add the core density without
    scaling parameter and 'scale' which will refine a scaling 
    parameter for the core density which might for systematic
    deviations due to a coarse valence density grid (untested!)
    By default 'constant'
    
  - `extinction`:
    
    Use an extinction correction. Options: 'none' -> no extinction
    correction, 'shelxl' use the (empirical) formula used by SHELXL 
    to correct to correct for extinction, 'secondary' see Giacovazzo
    et al. 'Fundmentals of Crystallography' (1992) p.97, by default
    'none'

  - `max_dist_recalc`:

    If the max difference in atomic positions is under this value in 
    Angstroems, no new structure factors will be calculated, by
    default 1e-6

  - `max_iter`:

    Maximum of refinement cycles if convergence not reached, by 
    default: 100

  - `min_iter`:

    Minimum refinement cycles. The refinement will stop if the
    wR2 increases if the current cycle is higher than min_iter,
    by default 10

  - `cutoff`:

    Expects a tuple of three values where the first two are strings and
    the last one is a float value. First string is a cutoff mode. 
    Currently available are `'none'` where all reflections are used,
    `'sin(theta)/lambda'` where the cutoff is set according to a user
    given resolution, 'fraction(f0jval)' where the resolution cutoff is
    set to include a certain fraction of the mean absolute *valence* 
    atomic form factors. `'I/esd(I)'` can be used for excluding values
    based on the value over estimated standard deviation
    The second string can be either be 'above' or 'below' and 
    denominates in which direction values will be excluded.
    The final value is the actual cutoff value.


In [ ]:
refinement_dict_caf2 = {
    'f0j_source': 'qe',
    'reload_step': 1,
    'core': 'constant'
}

The function in the next cell creates a list of parameters and a recipee of how to reconstruct the atomic parameters from these parameters (which might be non-trivial if we have contraints.)

In [ ]:
construction_instructions_caf2, parameters_caf2 = create_construction_instructions(
    atom_table=atom_table_caf2,
    constraint_dict=constraints_caf2,
    refinement_dict=refinement_dict_caf2
)

### Setting the options for the calculation
This dictionary is specific for each choice of `f0j_source` in the `refinement_dict`. It consists of some XHARPy specific options and then program specific options. Using your knowledge from the Quantum Espresso tutorial you might guess that the program specific options will be used to write an input file for `pw.x`. With some info added by XHARPy (for reference see [QE's pw.x input description](https://www.quantum-espresso.org/Doc/INPUT_PW.html)). More information about the individual computation dicts can be found in the [online documentation of XHARPy](https://xharpy.readthedocs.io/en/latest/library/library_f0j.html).

Unfortunately, XHARPy with QE can only work with lattices expressed as primitive (even if they are not, as in this case), as the coordinate transformation needed for centrings is not implemented.

In [ ]:
computation_dict_caf2 = {
    'symm_equiv': 'once', # XHARPy specific: f0j only evaluated once for every atom in asymmetric unit
    'mpicores': 2, # Sets the number of mpi cores. Make sure that n(MPI) * n(OMP) < N(Threads CPU)

    'control': {
        'prefix': 'CaF2',
        'pseudo_dir': './CaF2/pseudo/',
    },
    'system': {
        'ibrav': 1,
        'a': float(cell_caf2[0]),
        'ecutwfc': 100,
        'ecutrho': 400,
    },
    'paw_files': {
        'Ca': 'Ca.paw.upf',
        'F': 'F.paw.upf',
    },
    'k_points':{
        'mode': 'automatic',
        'input': '1 1 1 0 0 0'
    }
}

## Start the refinement
Now that we have assembled all the information and set all the options we can refine with atomic form factors calculated by Quantum Espresso.

In [ ]:
parameters_caf2, var_cov_mat_caf2, information_caf2 = refine(
    cell=cell_caf2, 
    symm_mats_vecs=symm_mats_vecs_caf2,
    hkl=hkl_caf2,
    construction_instructions=construction_instructions_caf2,
    parameters=parameters_caf2,
    wavelength=wavelength_caf2,
    refinement_dict=refinement_dict_caf2,
    computation_dict=computation_dict_caf2
)

## Writing the results back to disk
After we have done out calculation we of course want to write the results back to disk. For this, XHARPy relies a lot on pre-existing files from ShelXL, which it uses to complete its output files.

We can now write an fcf(4) file for publication

In [ ]:
write_fcf(
    fcf_path=output_folder_caf2 / 'xharpy.fcf',
    fcf_dataset='xharpy',
    fcf_mode=4,
    cell=cell_caf2,
    hkl=hkl_caf2,
    construction_instructions=construction_instructions_caf2,
    parameters=parameters_caf2,
    wavelength=wavelength_caf2,
    refinement_dict=refinement_dict_caf2,
    symm_strings=symm_strings_caf2,
    information=information_caf2,
);

### Writing an fcf(6) and SHELXL res for the difference electron density
Using template files we can also output a SHELXL format res and an fcf(6). We can use this to display the structure and the difference electron density in software like MolecoolQt

In [ ]:
write_fcf(
    fcf_path=output_folder_caf2 / 'xharpy_6.fcf',
    fcf_dataset='xharpy_6',
    fcf_mode=6,
    cell=cell_caf2,
    hkl=hkl_caf2,
    construction_instructions=construction_instructions_caf2,
    parameters=parameters_caf2,
    wavelength=wavelength_caf2,
    refinement_dict=refinement_dict_caf2,
    symm_strings=symm_strings_caf2,
    information=information_caf2,
);

In [ ]:
write_res(
    out_res_path=output_folder_caf2 / 'xharpy_6.res',
    in_res_path=folder_caf2 / 'CaF2.lst',
    cell=cell_caf2,
    cell_esd=cell_esd_caf2,
    construction_instructions=construction_instructions_caf2,
    parameters=parameters_caf2,
    wavelength=wavelength_caf2
)

### Writing a cif file and adding the difference electron density
The cif file is also generated using a template (to reduce the amount of finalising needed). The bond and angle table are used to generate updated tables for the new cif files. Because bonds to hydrogen atoms might be omitted in out IAM refinement, we get a warning telling us to check whether the absence of bonds with hydrogen as bonding partners is expected. (Which it is in this case).

In [ ]:
write_cif(
    output_cif_path=output_folder_caf2 / 'xharpy.cif',
    cif_dataset='xharpy',
    shelx_cif_path=folder_caf2 / 'CaF2.cif',
    shelx_dataset=0,
    cell=cell_caf2,
    cell_esd=cell_esd_caf2,
    symm_mats_vecs=symm_mats_vecs_caf2,
    hkl=hkl_caf2,
    construction_instructions=construction_instructions_caf2,
    parameters=parameters_caf2,
    var_cov_mat=var_cov_mat_caf2,
    refinement_dict=refinement_dict_caf2,
    computation_dict=computation_dict_caf2,
    information=information_caf2
)

XHARPy has no way to generate difference electron densities on its own. Instead it relies on cctbx. Unfortunately, that means we need to add the difference electron density entries using the fcf6 file.

In [ ]:
add_density_entries_from_fcf(output_folder_caf2 / 'xharpy.cif', str(output_folder_caf2 / 'xharpy_6.fcf'))

### Exporting a tsc File from a Refinement result
If we want to use the result in other software we can just export a [tsc file](https://arxiv.org/pdf/1911.08847.pdf) for our final refinement.

In [ ]:
from xharpy.io import f0j2tsc
f0j2tsc(
    file_name=output_folder_caf2 / 'xharpy.tsc',
    f0j=information_caf2['f0j_anom'],
    construction_instructions=construction_instructions_caf2,
    symm_mats_vecs=symm_mats_vecs_caf2,
    index_vec_h=hkl_caf2[['h', 'k', 'l']].values,
    remove_anom=True
)

## Writing a tsc for a given CIF
This code demonstrates two things: The direct generation of a tsc from a given cif and using gpaw_mpi as a backend. The options of the needed `export_dict` can be accessed by the in code documentation of cif2tsc. The main addition is the definition of a resolution limit up until which we generate the f0j. The `computation_dict` is identical to the one we would use for refinement using GPAW.

In [ ]:
help(cif2tsc)

In [ ]:
folder_urea = Path('./urea')
output_folder_urea = folder_urea / 'xharpy_output'
output_folder_urea.mkdir(exist_ok=True)

export_dict_urea = {
    'f0j_source': 'gpaw_mpi',
    'core': 'constant',
    'resolution_limit': 'cif'
}

computation_dict_urea = {
    'symm_equiv': 'once',
    'gridinterpolation': 4, 
    'xc': 'SCAN',
    'txt': output_folder_urea / 'gpaw.txt',
    'mode': 'fd',
    'h': 0.16,
    'convergence':{'density': 1e-7},
    'kpts': {'size': (1, 1, 1), 'gamma': True},
    'symmetry': {'symmorphic': False},
    'nbands': -2,
    'save_file': output_folder_urea / 'gpaw_result.gpw'
}

cif2tsc(
    tsc_path=output_folder_urea / 'xharpy.tsc',
    cif_path=folder_urea / 'iam.cif',
    cif_dataset=0,
    export_dict=export_dict_urea,
    computation_dict=computation_dict_urea
)

# What is different to other HAR implementations here?
Just as with the other programs for Hirshfeld atom refinement, such as Tonto and NoSpherA2, XHARPy partitions the densities of atoms. There are three differences to the other approaches. 
 1. The underlying calculation is periodic using projector augmented waves (PAW) with a plane wave (Quantum Espresso) or real-space grid (GPAW) basis.
 2. The valence density (not the pseudo density!) is expanded onto a rectangular grid, partitioned and then Fourier transformed via FFT
 3. The spherical frozen core density is evaluated on a non-uniform linear grid (more points at the centre) as spherically symmetry

Because the PAW bases are constructed via calculating a free atom, we use these free atoms as libraries for our atomic densities in Hirshfeld partitioning.

# Working outside of Jupyter

## A quick word about the CLI(s)

There is two command line interace scripts available with a very limited subset of the available functionality. At the moment these also only support GPAW. You can access them by typing the following command (with the XHARPy conda environment active).

```bash
python -m xharpy.cli_refine
python -m xharpy.cli_tsc
```
for doing a refinement or generating a tsc file respectively. You can pass the flag `--help` to get information on how to pass more information in a non-interactive mode.

## Exporting a Jupyter notebook as python script
You can directly download the code from a notebook as a python script using Jupyter's File menu. This makes things available for a less interactive session (say on a cluster)

# Final remarks
Just as in a LCAO refinement, you need to list the size of the used basis, when reporting for publications and presentations, as well as the employed functional.
 - For Quantum Espresso list the cutoff energies and the employed $k$-point grid.
 - For GPAW list the grid spacing (`h` in the `computation_dict`) and the employed $k$-point grid.
 
Currently unavailable is the refinement of spin-polarised systems (as in unpaired electrons).

The full code and more examples are available at: [https://github.com/Niolon/XHARPy](https://github.com/Niolon/XHARPy)